In [76]:
!pip install geopandas
!pip install pysal

In [ ]:
!pip install --ignore-installed geopandas

  Using cached https://files.pythonhosted.org/packages/5b/0c/e6c99e561b03482220f00443f610ccf4dce9b50f4b1093d735f93c6fc8c6/geopandas-0.6.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9b/9a/04dcffd4ff9bc064c05f723be71177cfb593d5ce6e4d6185c3ef8b7a4be9/pyproj-2.2.2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/81/d1/b8e1b089a8ddd6df74be583d70373eac55c725c6197c115efbd3c3e1509f/Shapely-1.6.4.post2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/db/83/7d4008ffc2988066ff37f6a0bb6d7b60822367dcb36ba5e39aa7801fda54/pandas-0.24.2-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/89/dd/289641061a69bb9b302e9fd52e5532f8345f6b277d8f4c98419fbbef1a14/Fiona-1.8.13-cp27-cp27mu-manylinux1_x86_64.whl


In [ ]:
!pip install pysal==2.0.0

In [ ]:
!pip install descartes

In [ ]:
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import pylab
import descartes
import numpy as np
#import seaborn as sns
#import time
#import os

%matplotlib inline

plt.style.use('ggplot')
pylab.rcParams['figure.figsize'] = (40., 32.)

In [ ]:
data = "data/borough_heatmap/London_Borough_Excluding_MHW.shp"

mapBoroughs = gp.read_file(data)
mapBoroughs.head()

In [ ]:
mapBoroughs.crs = {'init': u'epsg:4326'}
target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'merc'}
projected_mapBoroughs = mapBoroughs.to_crs(crs=target_crs)

In [ ]:
data_path = "./data/borough_heatmap/MPS Borough Level Crime (most recent 24 months).csv"

sexassaults = pd.read_csv(data_path)
sexassaults= sexassaults[sexassaults['MajorText']=='Sexual Offences']
sexassaults.head()

In [ ]:
sexassaults=sexassaults.set_index(['LookUp_BoroughName'])
sexassaults['2019']=(sexassaults['201901']+sexassaults['201902']+sexassaults['201903']+sexassaults['201904']
                     +sexassaults['201905']+sexassaults['201906']+sexassaults['201907']+sexassaults['201908']
                     +sexassaults['201909']+sexassaults['201910']+sexassaults['201911'])
sexassaults= sexassaults.drop(columns=['201712', '201802','201801','201802','201803','201804','201805',
                                       '201806','201807','201808','201809','201810','201811','201812',
                                       '201901','201902','201903','201904','201905','201906','201907',
                                       '201908','201909','201910','201911'])
sexassaults.head()

In [ ]:
total = sexassaults.groupby(['LookUp_BoroughName'])['2019'].sum()
sexassaults['Total'] = total
sexassaults.head()

In [ ]:
geosexassaults = mapBoroughs.merge(sexassaults, left_on= 'NAME', right_on= 'LookUp_BoroughName', how= 'inner')
geosexassaults

In [ ]:
cols = geosexassaults.columns.tolist()
cols = cols[-1:] + cols[:-1]
geosexassaults = geosexassaults [cols]
#changed column order for .sum .groupby need elemnt to be index and 2019 column to be right next one to itù
#geosexassaults

In [ ]:
original_crs = geosexassaults.crs
target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'merc'}
projected_geosexassaults = geosexassaults.to_crs(crs=target_crs)
geosexassaults

In [ ]:
#Install the geopandas module
!pip install --ignore-installed geopandas

#Uninstall any existing version; forcing Yes
!pip uninstall --yes pysal

#Install newest branch
!pip install pysal==1.14.4

In [ ]:
#to get the geographic information: 
original_crs = geosexassaults.crs
target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'merc'}
projected_geosexassaults = geosexassaults.to_crs(crs=target_crs)

# Convenience functions for working with colour ramps and bars
def colorbar_index(ncolors, cmap, labels=None, **kwargs):
    """
    This is a convenience function to stop you making off-by-one errors
    Takes a standard colour ramp, and discretizes it,
    then draws a colour bar with correctly aligned labels
    """
    cmap = cmap_discretize(cmap, ncolors)
    mappable = plt.cm.ScalarMappable(cmap=cmap)
    mappable.set_array([])
    mappable.set_clim(-0.5, ncolors+0.5)
    colorbar = matplotlib.pyplot.colorbar(mappable, **kwargs)
    colorbar.set_ticks(np.linspace(0, ncolors, ncolors))
    colorbar.set_ticklabels(range(ncolors))
    if labels:
        colorbar.set_ticklabels(labels)
    return colorbar

def cmap_discretize(cmap, N):
    """
    Return a discrete colormap from the continuous colormap cmap.

        cmap: colormap instance, eg. cm.jet. 
        N: number of colors.

    Example
        x = resize(arange(100), (5,100))
        djet = cmap_discretize(cm.jet, 5)
        imshow(x, cmap=djet)

    """
    if type(cmap) == str:
        cmap = get_cmap(cmap)
    colors_i = np.concatenate((np.linspace(0, 1., N), (0., 0., 0., 0.)))
    colors_rgba = cmap(colors_i)
    indices = np.linspace(0, 1., N + 1)
    cdict = {}
    for ki, key in enumerate(('red', 'green', 'blue')):
        cdict[key] = [(indices[i], colors_rgba[i - 1, ki], colors_rgba[i, ki]) for i in xrange(N+1)]
    return matplotlib.colors.LinearSegmentedColormap(cmap.name + "_%d" % N, cdict, 1024)

from pysal.esda.mapclassify import Quantiles

# to create and split values for the colorbar
breaks = Quantiles(
    geosexassaults['Total'].values,
    k=5)
bar_labels = ['<=%i'% b for b in breaks.bins]


geosexassaults.plot(column='Total', colormap='OrRd', k=5)
plt.title('London sexual offences 2019', \
              fontdict={'fontsize': '50',
                        'fontweight' : '3'})

# create an annotation for the  data source
plt.annotate('Source: London Datastore, 2014',
           xy=(0.1, .08), xycoords='figure fraction',
           horizontalalignment='left', verticalalignment='top',
           fontsize=20, color='#555555')

#remove axis
plt.axis('off')

cmap = plt.get_cmap('OrRd')
colorbar_index(ncolors=5, cmap=cmap, shrink=0.5, labels=bar_labels)
plt.savefig('./data/heat_map/2019map_sexassaults.png',bbox_inches = 'tight')